In [1]:
import time
import os
from IPython.display import clear_output

# Define Styles
from IPython.display import display
from IPython.core.display import HTML

with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# Play vs The AI Chess Engine
This notebook allows to play games against the chess engine. The depth can be manually set and the game results/log will be written into a logfile.

### Basic setup: running the notebook `chess_core` and creating a new `board`

In [ ]:
%run chess_core.ipynb
board = chess.Board()
board = board.root()
clear_output(wait=True)

### Change depth here if desired
Disclaimer: a depth above 5 or 6 (depending on the computer this notebook is running on) can result in much higher computation times, possibly making the game unplayable.

In [ ]:
depth = 5

### Change player color if desired (you have to run the "basic setup" again to restart the game)

In [ ]:
player_color = chess.WHITE

## Play against the AI
The following is the code to play against the AI. It can be divided into four parts:
 - preparing the log file
 - taking the player move and calculating/executing the AI move
 - printing and logging
 - checking for game over
 
First, the logfile for the recently started game is being created. For that reason, in case the directory does not exist yet, it will be created as well. After creating the logfile, the main `while` loop starts and always follows the same pattern: first, it takes the player move as a text input. **Note: the move has to follow the _long algebraic notation_ e.g. `d2d4` or `c3f6`**. If the move is illegal, the player has to input a new one. After that, the move will be executed and the resulting board will be passed to the `minimax_input` function, together with the earlier defined `depth`.  The move (as well as the value) will be returned by this function and the former will be executed. After that, many values for debugging and information purposes will be evaluated and printed: the overall time needed, the chosen move together with the value, and the amount of cache hits in relation to the overall number of calls of the minimax function. This data, together with the board (in _fen_ notation) will then also be written to the created logfile. Lastly, the function checks for a game over state and prints it, if present.

After this, the loop will start over again. This happens until the game is over, also using the `winning_state` function.

In [ ]:
%%time
skip_player_move = not player_color # Shold not skip a move for the player if the color is white - should skip once for black
player_had_last_move = False # Flag to correctly interpret the result of the winning_state function; if True, then the result will be calculated for the ai, thus creating an inverted result
display(board)
log_dir_name = ".logs"
filepath = os.path.join(log_dir_name,"log_" + str(time.time()) + ".log")
if not os.path.exists(log_dir_name):
    os.makedirs(log_dir_name)
with open(filepath,'a') as logfile:
    logfile.write("Depth: " + str(depth) + " - Player Color: " + ("WHITE" if player_color else "BLACK") + "\n\n")
    while winning_state(board) is None:
        if skip_player_move:
            skip_player_move = False
            player_move = ""
        else:
            print("Player Move:")
            player_move = input()
            player_uci_move = chess.Move.from_uci(player_move)
            if not player_uci_move in board.legal_moves:
                print("Illegal Move, try again")
                continue
            board.push(player_uci_move)
            clear_output(wait=True)
            display(board)
        start_time = time.time()
        if winning_state(board) is None:
            value, move = minimax_input(board, depth)
            board.push(move)
        else: # Game is already over, player had the last move and thus the result of winning_state shall be inverted
            player_had_last_move = True
        clear_output(wait=True)
        display(board)
        time_for_move = time.time() - start_time
        minimax_cache_ratio = CACHE_HITS / MINIMAX_CALLS * 100
        print("AI Time:", round(time_for_move, 2), "seconds")
        print("AI Move:\n", move, "| Value:", value)
        print("Minimax calls:", MINIMAX_CALLS)
        print("CACHE_HITS:", CACHE_HITS, "(", (minimax_cache_ratio), "%)")
        print(f"Total Cache items: {len(CACHE)}")
        logfile.write("Player move:" + str(player_move) +
                      " | FEN: " + str(board.fen()) +
                      "\nAI move: " + str(move) + 
                      " | Projected value: " + str(value) + 
                      " | Current value: " + str(static_eval(board,is_endgame(board))) +
                      " | Time: " + str(time_for_move) + " seconds." +
                      "\nMinimax accesses: " + str(MINIMAX_CALLS) +  
                      " | Cache accesses: " + str(CACHE_HITS) + 
                      " | Ratio: " + str(minimax_cache_ratio) + "%" + 
                      "\n==================== \n")
        win_state = winning_state(board)
        if win_state == None:
            pass
        elif win_state == 0:
            print("=====Game over, draw!=====")
            logfile.write("Draw")
        elif win_state < 0:
            if player_had_last_move:
                print("=====Game over, you won!=====")
                logfile.write("AI lost")
            else:
                print("=====Game over, you lost!=====")
                logfile.write("AI won")
print("Log has been written to", filepath)